![Сбор данных](https://www.shkolazhizni.ru/img/content/i204/204002_or.jpg)

# Прогнозирование стоимости автомобиля по его характеристикам
Часть 1
__________________________________________________________________________________________________________________

**Эта часть проекта включает:**
> - анализ структуры сайта [auto.ru](https://auto.ru/) на предмет поиска тэгов и классов, хранящих требуемую для проекта информацию;
> - сбор ссылок с внешнего сайта [auto.ru](https://auto.ru/) на отдельные объявления по продаже б-у автомобилей;
> - сбор данных по каждой ссылке, собранной на первом шаге;
> - использовнаие библиотеки BeautifulSoup для поиска по тэгам и классам необходимой для проекта информации;
> - запись собранных данных в промежуточный файл для дальнейшей обработки собранных данных;

# Загрузка библиотек

In [1]:
#библиотеки, необходимые для формирования запросов сайту и получение ответов
#и поиска в полученной с сайта информации необхоимых для проекта данных
from bs4 import BeautifulSoup
import requests as req
import lxml.html as html
import urllib3

import time #библиотека для установки паузы между запросами к сайту

#общие библиотеки
import pandas as pd
import numpy as np

# Блок сбора данных с сайта

In [8]:
#формируем header для включения его в запросы, так как сайты могут блокировать запросы без него
header = {'user-agent': 'Edg/95.0.1020.53'}
urllib3.disable_warnings()

#сайт имеет постраничную структуру
#поэтому формируем стартовую ссылку, к которой будем прибавлять страницы
#и формировать список ссылок для конкретного бренда и модели,
# каждая из которых будет содержать примерно 38 ссылок на конкретные автомобили
#всего в тестовом датасете 12 моделей, по ним будем собирать информацию
url_start = 'https://auto.ru/cars/audi/a4/used/'
urls = []

for i in range(5,7): #количество страниц задавалось в зависимости от количества страниц на сайте авто.ру для данной модели
    urls.append(url_start+'?page='+str(i))

In [10]:
#формируем пустой список и заполняем его собранными на автомобили ссылками
urls_auto = []
for url in urls:
    response = req.get(url, headers=header, timeout=50, verify=False)
    page = BeautifulSoup(response.text,'html.parser')
    for a in page.find_all('a', class_="ListingItemTitle__link"):
        urls_auto.append(a['href'])
    time.sleep(3)

In [11]:
#проверяем сколько собрали ссылок
len(urls_auto)

74

In [12]:
#создаем пустой массив для заполнения его собранными данными
data_array_full = np.empty(15, 'str')

#формируем списки с тэгами и классами, по которым будем искать нужную нам информацию
tags = ['li','h1','li','li','li','li','li','li','li','li','li','li','li','li','div']
classes = ["CardInfoRow CardInfoRow_bodytype", "CardHead__title", "CardInfoRow CardInfoRow_kmAge",
           "CardInfoRow CardInfoRow_color", "CardInfoRow CardInfoRow_engine",
           "CardInfoRow CardInfoRow_complectationOrEquipmentCount", "CardInfoRow CardInfoRow_transportTax",
           "CardInfoRow CardInfoRow_transmission", "CardInfoRow CardInfoRow_drive", "CardInfoRow CardInfoRow_wheel",
           "CardInfoRow CardInfoRow_ownersCount", "CardInfoRow CardInfoRow_customs",
           "CardInfoRow CardInfoRow_pts", "CardInfoRow CardInfoRow_year", "PriceUsedOffer"]

#формируем цикл, который пробегается по собранному на предыдущем шаге списку ссылок с объявлениями
#собирает данные по каждой ссылке пробегая список всех пар тэг-класс
#в запросах библиотеки request будем использовать команды try-except, так, чтобы ошибки не останавливали процесс цикла
#в случае ошибок в список данных будем добавлять NaN
for urls in urls_auto:
    try:
        response = req.get(urls, headers=header, timeout=50, verify=False)
        page = BeautifulSoup(response.text,'html.parser')
    except:
        print('ошибка запроса')
    
    data = []
    
    for i in range(15):
        try:
            feature = page.find_all(tags[i], class_=classes[i])
            feature = feature[0].text.encode('l1').decode()
        except:
            feature = np.NaN
        
        data.append(feature)
        
    data_array = np.asarray(data).reshape(1,15)
    data_array_full = np.row_stack((data_array_full,data_array))
    
    time.sleep(3) #поставим паузу 3 секунды между запросами во избежание перегрузки сайта

In [13]:
#формируем дата фрейм из собранных данных
#порядок колонок соответствует порядку тэгов и классов
# названия колонок составим аналогично колонкам в тестовом сете данных
columns = np.array(['bodytype','brand','mileage','color', 'engine', 'complectation',
                    'tax', 'transmission', 'drive', 'wheel','owners', 'customs', 'pts',
                    'year','price'])
data_auto = pd.DataFrame(data=data_array_full, columns=columns)
data_auto.replace('nan', np.NaN, inplace=True)

In [14]:
#смотрим и проверяем собранную структуру данных
#данные годятся, но они очень сырые и требуют значительной предобработки перед тем, как передавать их в моделирование
data_auto.sample(3)

,bodytype,brand,mileage,color,engine,complectation,tax,transmission,drive,wheel,owners,customs,pts,year,price
70,Кузовседан,Audi A4 V (B9),Пробег85 883 км,Цветсеребристый,Двигатель2.0 л / 249 л.с. / Бензин,Комплектация23 опции,Налог18 675 ₽ / год,Коробкароботизированная,Приводполный,РульЛевый,Владельцы1 владелец,ТаможняРастаможен,ПТСОригинал,год выпуска2018,2 490 000 ₽Хорошая ценаот 2 370 000 ₽ со скидками
13,Кузовседан,Audi A4 II (B6),Пробег348 000 км,Цветсерый,Двигатель1.8 л / 150 л.с. / Бензин,Комплектация47 опций,Налог5 100 ₽ / год,Коробкавариатор,Приводпередний,РульЛевый,Владельцы3 или более,ТаможняРастаможен,ПТСДубликат,год выпуска2001,350 000 ₽Смотреть статистику цен
67,Кузовседан,Audi A4 IV (B8),Пробег255 000 км,Цветзелёный,Двигатель2.7 л / 190 л.с. / Дизель,Комплектация23 опции,Налог9 500 ₽ / год,Коробкавариатор,Приводпередний,РульЛевый,Владельцы1 владелец,ТаможняРастаможен,ПТСОригинал,год выпуска2009,1 230 000 ₽Смотреть статистику цен


In [15]:
#сохраняем данные в промежуточном буферном файле
data_auto.to_csv('C:/Users/Алексей Третьяков/Desktop/ML_projects/FierceDra-X.github.io/module_5/data_auto_f.csv', index=False)

 ```
 Заканчиваем эту часть копированием собранных данных в промежуточный файл data_auto_f.csv, данные в котором потом будут обработаны до необходимого для моделирования качества во второй части проекта - в файле data_auto_processing.ipynb.
  ```